Lendo Arquivos da Aula anterior

In [0]:
%fs
ls 'dbfs:/databricks-datasets/iot/'

In [0]:
%sql
select * from json.`dbfs:/databricks-datasets/iot/iot_devices.json`

Copiar este arquivo para nosso DBFS 

In [0]:
%python

dbutils.fs.cp(
    'dbfs:/databricks-datasets/iot/iot_devices.json',
    'dbfs:/mnt/iot/iot_devices.json'
)



In [0]:
%python
display(dbutils.fs.ls('dbfs:/mnt/iot/'))

In [0]:
%sql
-- consulta em arquivo JSON
select * from json.`dbfs:/mnt/iot/iot_devices.json`

In [0]:
%sql
-- consulta em arquivo JSON
select * from json.`dbfs:/mnt/iot/*.json`

Transformando arquivo em tabela

In [0]:
%python
display(dbutils.fs.ls('dbfs:/mnt/iot/'))

In [0]:
%sql
-- Cria uma tabela a partir do arquivo JSON com comando Using 
CREATE TABLE iot_devices
USING json
LOCATION 'dbfs:/mnt/iot/';

-- Conta o número de registros na tabela criada
SELECT COUNT(*) FROM iot_devices;

In [0]:
select * from iot_devices

In [0]:
%sql
-- Ver informações completas da tabela 
DESCRIBE EXTENDED iot_devices

Se eu adicionar um Arquivo novo na pasta isso vai atualizar minha tabela?

In [0]:
SELECT COUNT(*) FROM iot_devices;

Novos Arquivos sao armazenados na pasta de origem da tabela  em Cache  da operação , e a leitura é em Cache local desta forma a tabela nao é atualizada com os novos dados , vamor ver  na prática?
a tabela de origem  Arquivo json nao esta configurado para mostrar que ele deve ser sempre atualizado , mas podemos atualizar manualmente 


In [0]:
%python
dbutils.fs.cp(
    'dbfs:/databricks-datasets/iot/iot_devices.json',
    'dbfs:/mnt/iot/iot_devices2.json'
)


In [0]:
%python
display(dbutils.fs.ls('dbfs:/mnt/iot/'))

In [0]:
SELECT COUNT(*) FROM iot_devices;

In [0]:
REFRESH table iot_devices

REFRESH TABLE  o que faz este comando?

https://learn.microsoft.com/pt-pt/azure/databricks/sql/language-manual/sql-ref-syntax-aux-cache-refresh-table


Revalida todos os dados em Cash no diretório e atualiza a tabela OBS: Para um grande volume de dados esta operação pode demorar , talvez sendo inviavel a ultilização deste recurso em produção


#### Impactos de não usar o formato Delta ao criar tabelas

Criar tabelas usando formatos como JSON ou CSV em vez do formato Delta pode ter vários impactos, incluindo:

- **Desempenho**: O formato Delta é otimizado para leitura e escrita rápidas, enquanto formatos como JSON e CSV podem ser mais lentos devido à falta de otimizações como indexação e compactação.

- **Transações ACID**: O formato Delta oferece suporte a transações ACID, garantindo consistência e isolamento de dados. Formatos como JSON e CSV não oferecem essas garantias.

- **Gerenciamento de Versões**: O formato Delta permite o versionamento de dados, facilitando operações como rollback e time travel. Isso não é possível com JSON ou CSV.

- **Atualizações e Exclusões**: O formato Delta permite operações de atualização e exclusão eficientes. Com JSON e CSV, essas operações são mais complexas e menos eficientes.

- **Então quando vou usar isso?**: Em ambientes de teste quer ler um arquivo rápido para ver seu conteúdo , o que deve tratar, para ja preparar para projetos futuros antes de criar todo seu Pipiline de dados ou informações gerenciais , mostrar em reunião antes de inserir em produção (vou te dar um exemplo)

Criando tabela Delta a partir de arquivo com CREATE TABLE AS (CTAS)

In [0]:
-- Comandos CTA  inferem os tipos de dados altomaticamente
-- Desafio: Criar tabela iot_devices_dt 
CREATE TABLE iot_devices_dt
AS
SELECT * FROM json.`dbfs:/mnt/iot/*.json`


In [0]:
SELECT * FROM iot_devices_dt

In [0]:
SELECT COUNT(*) FROM iot_devices_dt

In [0]:
DESCRIBE EXTENDED iot_devices_dt

In [0]:
%python
# Copiar o arquivo para o seu DBFS
dbutils.fs.cp(
    'dbfs:/databricks-datasets/iot/iot_devices.json',#origem
    'dbfs:/mnt/iot/iot_devices3.json' #destino
)

In [0]:
%python
display(dbutils.fs.ls('dbfs:/mnt/iot/'))

In [0]:
SELECT COUNT(*) FROM iot_devices_dt

Final se prescisar apagar

In [0]:
DROP TABLE IF EXISTS default.iot_devices;
DROP TABLE IF EXISTS default.iot_devices_dt;

In [0]:
%fs
rm -r 'dbfs:/mnt/iot/'